In [1]:
import numpy as np
import tensorflow as tf
import collections
from konlpy.tag import Twitter
import re

C:\Users\easts\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
with open('../grimm-split.txt', encoding = 'utf-8') as f:
    texts = f.readlines()

desc_sentence = []
words = []
for desc in texts[:3]:
    desc_text = desc.split('\t')[1]
    #print(desc_text)
    for desc_line in desc_text.split('|'):
        desc_sentence.append(desc_line)
        sentence = re.sub(r"[^ㄱ-힣a-zA-Z0-9]+", ' ', desc_line).strip().split()
        if sentence:
            words.append(sentence)
            print(sentence)


['옛날', '옛적에', '소망을', '이루어', '주는', '힘이', '여전히', '통했던', '때에', '딸들이', '모두', '예쁜', '왕이', '한', '분', '살고', '계셨어요']
['특히나', '막내공주님이', '아주', '예뻤는데요', '어찌나', '아름다운지', '해도', '그녀의', '얼굴에', '햇볕을', '비출', '때마다', '감탄을', '하곤', '하였더랬죠']
['왕의', '성은', '울창한', '숲에', '둘러싸여', '있었어요']
['숲에', '있는', '나이', '많은', '라임', '나무', '아래에', '우물이', '하나', '있었어요']
['화창한', '날에', '막내공주님께서', '숲으로', '놀러', '나와', '시원한', '분수', '옆에', '앉았어요']
['그녀는', '따분할', '땐', '황금', '공을', '가져와', '높이', '던졌다가', '다시', '잡곤', '했지요']
['황금', '공놀이는', '그녀가', '가장', '좋아하는', '놀이었어요']
['그러다', '우연히', '공주의', '작은', '손으로', '떠받치고', '있던', '황금', '공이', '바닥으로', '떨어져', '곧장', '우물', '속으로', '굴러', '들어갔지', '뭐예요']
['공주님이', '다급히', '눈으론', '공을', '쫓았지만', '허사였어요']
['우물은', '꽤', '깊었어요', '바닥이', '보이지', '않을', '정도로요']
['이', '일로', '공주님이', '울기', '시작했어요', '소리', '내', '울고', '더', '크게', '소리', '내', '울었어요', '마음의', '안정을', '찾을', '수가', '없었어요']
['그렇게', '눈물을', '글썽이고', '있는데', '누군가가', '그녀에게', '이렇게', '말하는', '거예요']
['누가', '널', '괴롭히니', '공주야', '네', '모습을', '보고', '돌도', '눈물을', '뚝뚝', '흘리겠다']
['공주가', '목소리가', '들려

In [8]:
# 단어의 빈도수를 세고 빈도수가 일정 이상인 단어만 저장한다.
word_counter = [['UNK', -1]] # 빈도수 문제로 word_dict에 없는 word를 처리하기 위함. unknown
word_counter.extend(collections.Counter([word for sentence in words for word in sentence]).most_common())
print(word_counter)
word_counter = [item for item in word_counter if item[1] >= 0 or item[0] == 'UNK']
print(word_counter)

# 저장한 단어의 index를 지정하고 저장.
word_dict = dict()
for word, count in word_counter:
    word_dict[word] = len(word_dict)
word_reverse_dict = dict(zip(word_dict.values(), word_dict.keys()))
print(len(word_dict))
print(word_dict)
print(word_reverse_dict)

[['UNK', -1], ('그', 37), ('말했어요', 33), ('하지만', 28), ('한', 26), ('라며', 25), ('네', 24), ('마리아께서', 18), ('개구리가', 17), ('고양이가', 17), ('내', 16), ('수', 16), ('다', 16), ('공주가', 15), ('다시', 15), ('성모', 14), ('이', 14), ('있는', 13), ('그래서', 13), ('공을', 12), ('소녀가', 12), ('더', 12), ('방문을', 12), ('내가', 12), ('그리고', 11), ('소녀는', 11), ('자신의', 11), ('네가', 11), ('황금', 11), ('쥐가', 10), ('라고', 10), ('게', 10), ('왕비가', 10), ('예쁜', 10), ('아기', 10), ('왕이', 9), ('세', 9), ('그러자', 9), ('동정녀', 9), ('거야', 9), ('했어요', 8), ('말씀하셨어요', 8), ('고양이는', 8), ('문을', 8), ('앉아', 8), ('날', 7), ('왕비의', 7), ('함께', 7), ('다음', 7), ('아이를', 7), ('있었어요', 7), ('번', 7), ('금지된', 7), ('든', 7), ('가', 7), ('넌', 7), ('대답했어요', 7), ('그런', 7), ('두', 7), ('그녀가', 7), ('말을', 6), ('그녀를', 6), ('항아리를', 6), ('너무도', 6), ('그녀의', 6), ('그래', 6), ('자신이', 6), ('거예요', 6), ('않았어요', 6), ('나무', 6), ('쥐는', 6), ('돼지비계가', 6), ('데리고', 6), ('왕비를', 6), ('이번엔', 6), ('만약', 5), ('이제', 5), ('그건', 5), ('건', 5), ('다음날', 5), ('안', 5), ('하늘나라의', 5), ('싹', 5), ('아기를', 5), ('

In [9]:
# 단어들을 형태소 분석기로 분리한다.
word_to_pos_li = dict()
pos_list = list()
twitter = Twitter()
for w in word_dict:
    w_pos_li = list()
    for pos in twitter.pos(w, norm=True):
        w_pos_li.append(pos)
   
    # 단어별로 형태소 분리하고 원래의 index에 맞게 저장.
    word_to_pos_li[word_dict[w]] = w_pos_li
    pos_list += w_pos_li

# print(word_to_pos_li)
# print("pos_list: ", pos_list)

# 분리된 단어들 counting. (굳이 카운팅하는게 필요한지?)
pos_counter = collections.Counter(pos_list).most_common()
# print(pos_counter)

# 형태소 분석기에 의해 분리된 단어들을 각각 index를 지정하고 저장.
pos_dict = dict()
for pos, _ in pos_counter:
    pos_dict[pos] = len(pos_dict)

pos_reverse_dict = dict(zip(pos_dict.values(), pos_dict.keys()))

print(len(pos_dict))
print("pos_dict : ", pos_dict)
print("pos_reverse_dict : ", pos_reverse_dict)

C:\Users\easts\Anaconda3\envs\tensorflow-gpu\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


1547
pos_dict :  {('씻겨', 'Verb'): 311, ('문질러도', 'Verb'): 312, ('로움', 'Noun'): 313, ('났고', 'Verb'): 314, ('자꾸', 'Noun'): 315, ('불행', 'Noun'): 162, ('착석', 'Noun'): 316, ('물속', 'Noun'): 918, ('옛날', 'Noun'): 318, ('왕비', 'Noun'): 32, ('소리', 'Noun'): 18, ('해서', 'Verb'): 91, ('나이', 'Noun'): 319, ('없다', 'Adjective'): 320, ('연속', 'Noun'): 321, ('없어', 'Adjective'): 322, ('않아', 'Verb'): 163, ('주었어요', 'Verb'): 323, ('갑자기', 'Noun'): 325, ('든', 'Verb'): 326, ('두면', 'Verb'): 327, ('봤어요', 'Verb'): 328, ('있거라', 'Adjective'): 329, ('이번', 'Noun'): 92, ('어보', 'Noun'): 93, ('가시겠소', 'Verb'): 330, ('우울하게', 'Adjective'): 331, ('딸애', 'Noun'): 332, ('쥐', 'Noun'): 42, ('돌아온', 'Verb'): 334, ('둘러싸여', 'Verb'): 921, ('그', 'Determiner'): 94, ('이상한', 'Adjective'): 336, ('담은', 'Verb'): 337, ('먹어치어', 'Verb'): 338, ('돌아오셨어요', 'Verb'): 339, ('머리카락', 'Noun'): 340, ('성은', 'Noun'): 1488, ('쪽', 'Noun'): 164, ('붙잡아주기로', 'Verb'): 341, ('일로', 'Noun'): 342, ('합니다', 'Verb'): 343, ('겨', 'Noun'): 344, ('번째', 'Suffix'): 165, ('더랬', '

In [10]:

# 분리된 단어들을 word vector의 index로 매핑하여 분리되기전 index로 저장.
word_to_pos_dict = dict()
print(word_to_pos_li)

for word_id, pos_li in word_to_pos_li.items():
    print(word_id, ' ', pos_li)
    pos_id_li = list()
    for pos in pos_li:
        pos_id_li.append(pos_dict[pos])
    word_to_pos_dict[word_id] = pos_id_li
print(word_to_pos_dict)

{0: [('UNK', 'Alpha')], 1: [('그', 'Noun')], 2: [('말', 'Noun'), ('했어요', 'Verb')], 3: [('하지만', 'Conjunction')], 4: [('한', 'Verb')], 5: [('라며', 'Noun')], 6: [('네', 'Noun')], 7: [('마리아', 'Noun'), ('께서', 'Josa')], 8: [('개구리', 'Noun'), ('가', 'Josa')], 9: [('고양이', 'Noun'), ('가', 'Josa')], 10: [('내', 'Noun')], 11: [('수', 'Noun')], 12: [('다', 'Adverb')], 13: [('공주', 'Noun'), ('가', 'Josa')], 14: [('다시', 'Noun')], 15: [('성모', 'Noun')], 16: [('이', 'Noun')], 17: [('있는', 'Adjective')], 18: [('그래서', 'Adverb')], 19: [('공', 'Noun'), ('을', 'Josa')], 20: [('소녀', 'Noun'), ('가', 'Josa')], 21: [('더', 'Noun')], 22: [('방문', 'Noun'), ('을', 'Josa')], 23: [('내', 'Noun'), ('가', 'Josa')], 24: [('그리고', 'Conjunction')], 25: [('소녀', 'Noun'), ('는', 'Josa')], 26: [('자신', 'Noun'), ('의', 'Josa')], 27: [('네', 'Noun'), ('가', 'Josa')], 28: [('황금', 'Noun')], 29: [('쥐', 'Noun'), ('가', 'Josa')], 30: [('라고', 'Josa')], 31: [('게', 'Noun')], 32: [('왕비', 'Noun'), ('가', 'Josa')], 33: [('예쁜', 'Adjective')], 34: [('아기', 'Noun')], 35: 

763   [('13', 'Number'), ('개', 'Noun'), ('의', 'Josa')]
764   [('밀어줘요', 'Verb')]
765   [('아기', 'Noun'), ('천사', 'Noun'), ('들', 'Suffix'), ('과', 'Josa')]
766   [('연속', 'Noun')]
767   [('여', 'Noun'), ('는', 'Josa')]
768   [('잤어요', 'Verb')]
769   [('쾅', 'Noun')]
770   [('합니다', 'Verb')]
771   [('마찬가지', 'Noun'), ('로', 'Josa')]
772   [('지나고', 'Verb')]
773   [('아빠', 'Noun')]
774   [('안아', 'Verb')]
775   [('세례식', 'Noun'), ('에서', 'Josa')]
776   [('끝', 'Noun'), ('에', 'Josa')]
777   [('찾아', 'Verb')]
778   [('너', 'Noun'), ('도', 'Josa')]
779   [('외', 'Noun'), ('에는', 'Josa')]
780   [('담들', 'Verb')]
781   [('가도', 'Noun')]
782   [('역정', 'Noun'), ('을', 'Josa')]
783   [('때문', 'Noun'), ('이에요', 'Josa')]
784   [('되', 'Verb'), ('살아났고', 'Verb')]
785   [('그', 'Noun'), ('들', 'Suffix'), ('에게는', 'Josa')]
786   [('가증', 'Noun'), ('스런', 'Josa')]
787   [('삼켜', 'Verb'), ('버렸어요', 'Verb')]
788   [('빛남을', 'Verb')]
789   [('삼아죠', 'Verb')]
790   [('항아리', 'Noun'), ('는', 'Josa')]
791   [('구', 'Noun'), ('하지', 'Verb')]
792   [('

In [11]:
# Sub-sampling frequent words according to sampling_rate
# 빈번하게 나오는 단어를 희석한다. 전체 counter가 많아야 의미가 있는것 같음

def sub_sampling(data, word_counter, word_dict, sampling_rate):
    total_words = sum([len(sentence) for sentence in data])
    # print("total_words: {}".format(total_words))
    prob_dict = dict()
    for word, count in word_counter:
        f = count / total_words # 빈도수가 많을수록 f가 1에 가까워짐.
        p = max(0, 1 - math.sqrt(sampling_rate / f)) # sampling_rate가 0.0001이면 f가 클수록 prob이 커진다.
        prob_dict[word_dict[word]] = p
        # print("count : {}, f : {}, p : {}, prob_dict : {}".format(count, f, p, prob_dict))

    new_data = list()
    for sentence in data:
        s = list()
        for word in sentence:
            prob = prob_dict[word]
            if random.random() > prob: # prob이 작을수록 s에 저장되기 쉬움.
                s.append(word)
        new_data.append(s)

    return new_data

In [12]:
#
data = list()
unk_count = 0
for sentence in words:
    s = list()
    for word in sentence:
        print(word)
        if word in word_dict:
            index = word_dict[word]
        else:
            index = word_dict['UNK']
            unk_count += 1
        s.append(index)
        print(s)
    data.append(s)
print("before : ", word_counter)
word_counter[0][1] = max(1, unk_count)
print("after : ", word_counter)
print(data)

# data = sub_sampling(data, word_counter, word_dict, 0.0001)
# print(data)

옛날
[1177]
옛적에
[1177, 1605]
소망을
[1177, 1605, 1423]
이루어
[1177, 1605, 1423, 482]
주는
[1177, 1605, 1423, 482, 1651]
힘이
[1177, 1605, 1423, 482, 1651, 643]
여전히
[1177, 1605, 1423, 482, 1651, 643, 97]
통했던
[1177, 1605, 1423, 482, 1651, 643, 97, 1436]
때에
[1177, 1605, 1423, 482, 1651, 643, 97, 1436, 1040]
딸들이
[1177, 1605, 1423, 482, 1651, 643, 97, 1436, 1040, 576]
모두
[1177, 1605, 1423, 482, 1651, 643, 97, 1436, 1040, 576, 162]
예쁜
[1177, 1605, 1423, 482, 1651, 643, 97, 1436, 1040, 576, 162, 33]
왕이
[1177, 1605, 1423, 482, 1651, 643, 97, 1436, 1040, 576, 162, 33, 35]
한
[1177, 1605, 1423, 482, 1651, 643, 97, 1436, 1040, 576, 162, 33, 35, 4]
분
[1177, 1605, 1423, 482, 1651, 643, 97, 1436, 1040, 576, 162, 33, 35, 4, 1064]
살고
[1177, 1605, 1423, 482, 1651, 643, 97, 1436, 1040, 576, 162, 33, 35, 4, 1064, 121]
계셨어요
[1177, 1605, 1423, 482, 1651, 643, 97, 1436, 1040, 576, 162, 33, 35, 4, 1064, 121, 255]
특히나
[1654]
막내공주님이
[1654, 553]
아주
[1654, 553, 431]
예뻤는데요
[1654, 553, 431, 1342]
어찌나
[1654, 553, 431, 1342, 89

[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24]
너
[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24, 1215]
쥐는
[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24, 1215, 70]
특히
[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24, 1215, 70, 1234]
더
[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24, 1215, 70, 1234, 21]
조심해야해
[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24, 1215, 70, 1234, 21, 597]
사방에
[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24, 1215, 70, 1234, 21, 597, 1633]
위험투성이라
[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24, 1215, 70, 1234, 21, 597, 1633, 914]
어느
[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24, 1215, 70, 1234, 21, 597, 1633, 914, 225]
날
[3, 1720, 1412, 1469, 1477, 169, 163, 1586, 1132, 30, 9, 60, 527, 24, 1215, 70, 1234, 21, 597, 1633, 914, 225, 45]
쥐덫에
[3, 1720, 1412, 1469, 1477, 169, 1

누구요
[1308, 493]
누군데
[1308, 493, 506]
이
[1308, 493, 506, 16]
거친
[1308, 493, 506, 16, 108]
들판
[1308, 493, 506, 16, 108, 380]
여기에
[1308, 493, 506, 16, 108, 380, 1629]
앉아
[1308, 493, 506, 16, 108, 380, 1629, 44]
계신
[1308, 493, 506, 16, 108, 380, 1629, 44, 1382]
거요
[1308, 493, 506, 16, 108, 380, 1629, 44, 1382, 1348]
하지만
[3]
그녀는
[3, 92]
어떤
[3, 92, 231]
말도
[3, 92, 231, 407]
하지
[3, 92, 231, 407, 229]
않았어요
[3, 92, 231, 407, 229, 68]
왜냐하면
[3, 92, 231, 407, 229, 68, 155]
입을
[3, 92, 231, 407, 229, 68, 155, 203]
열
[3, 92, 231, 407, 229, 68, 155, 203, 1785]
수
[3, 92, 231, 407, 229, 68, 155, 203, 1785, 11]
업었기
[3, 92, 231, 407, 229, 68, 155, 203, 1785, 11, 1192]
때문이죠
[3, 92, 231, 407, 229, 68, 155, 203, 1785, 11, 1192, 210]
왕이
[35]
계속
[35, 646]
말했어요
[35, 646, 2]
나와
[160]
함께
[160, 47]
내
[160, 47, 10]
성으로
[160, 47, 10, 188]
가시겠소
[160, 47, 10, 188, 503]
그때
[161]
그녀가
[161, 59]
고개를
[161, 59, 119]
살짝
[161, 59, 119, 228]
끄떡였어요
[161, 59, 119, 228, 1125]
왕이
[35]
그녀를
[35, 61]
팔로
[35, 61, 1433]
안아
[35, 61, 143

In [14]:

print(word_reverse_dict[10])
print(word_to_pos_dict[10])
print(pos_reverse_dict[308])

내
[308]
('내', 'Noun')
